In [1]:
# UT-TOR-DATA-PT-01-2020-U-C Week 12 Homework
# Web Scraping Challenge
# (C) Boris Smirnov

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import re
import time

## [NASA Mars News](https://mars.nasa.gov/news/)

In [3]:
nasa_mars_news_url = 'https://mars.nasa.gov/news/'
browser = Browser('chrome', executable_path='chromedriver.exe', headless=False)
browser.visit(nasa_mars_news_url)

In [4]:
first_news_node = browser.find_by_css('li.slide').first

In [5]:
news_title = first_news_node.find_by_css('div.content_title').text
news_para = first_news_node.find_by_css('div.article_teaser_body').text

In [6]:
news_title

"NASA's Perseverance Rover Will Look at Mars Through These 'Eyes'"

In [7]:
news_para

'A pair of zoomable cameras will help scientists and rover drivers with high-resolution color images.'

In [8]:
browser.quit()

## [JPL Mars Space Images - Featured Image](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars)

N.B. Featured image changes periodically and it isn't necesserily a Mars image, it may be pretty much anything - Saturn, for example.

In [33]:
base_jpl_url = 'https://www.jpl.nasa.gov'
jpl_mars_images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser = Browser('chrome', executable_path='chromedriver.exe', headless=False)
browser.visit(jpl_mars_images_url)

In [38]:
featured_node = browser.find_by_css('article.carousel_item').first
s = featured_node['style']
featured_image_url = base_jpl_url + re.search(r'url\("(.+)"\)', s).group(1)
featured_image_alt = featured_node['alt']

In [39]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19113-1920x1200.jpg'

In [40]:
featured_image_alt

'Martian Concretions Near Fram Crater'

In [41]:
browser.quit()

## [Mars Weather](https://twitter.com/marswxreport?lang=en)

In [13]:
twitter_logout_url = 'https://twitter.com/logout'
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser = Browser('chrome', executable_path='chromedriver.exe', headless=False)
browser.visit(twitter_logout_url)

In [14]:
time.sleep(2)
browser.visit(mars_weather_url)
# Wait for the page to load!
# It didn't work without the delay
time.sleep(5)

In [15]:
no_weather_msg = "Failed to extract weather information"
mars_weather = no_weather_msg

# Elaborate approach: going down the DOM tree...
soup = BeautifulSoup(browser.html, 'lxml')
tweet_nodes = soup.select('div[data-testid="tweet"]')

for tweet_node in tweet_nodes:
    # tweet_node has 2 child nodes: [0] - left sidebar, [1] - right tweet body
    tweet_right_part = tweet_node.contents[1]

    # tweet_right_part has 2 child nodes: [0] - header part, [1] - tweet contents part
    tweet_contents_part = tweet_right_part.contents[1]

    # tweet_contents_part has 3 child nodes: [0] - text, [1] - image, [2] - controls (reply/retweet/like)
    tweet_text = tweet_contents_part.contents[0]

    mars_weather = tweet_text.text
    match = re.search(r'^InSight', mars_weather)
    if match:
        mars_weather = mars_weather.replace('\n', ' ')
        break


In [16]:
mars_weather

'InSight sol 508 (2020-05-01) low -92.2ºC (-134.0ºF) high -2.4ºC (27.7ºF) winds from the SW at 5.1 m/s (11.3 mph) gusting to 15.8 m/s (35.3 mph) pressure at 6.80 hPa'

In [17]:
if mars_weather == no_weather_msg:
    # Try again
    browser.reload()
    time.sleep(5)

    # Very stupid and straightforward (and probably the most effective) approach:
    # Just find the first string that looks like Mars weather
    match = re.search(r'InSight ([^<]+) hPa', browser.html)
    if match: # Gotcha!
        s = match.group(1)
        mars_weather = 'InSight ' + s.replace('\n', ' ') + ' hPa'


In [18]:
mars_weather

'InSight sol 508 (2020-05-01) low -92.2ºC (-134.0ºF) high -2.4ºC (27.7ºF) winds from the SW at 5.1 m/s (11.3 mph) gusting to 15.8 m/s (35.3 mph) pressure at 6.80 hPa'

In [19]:
browser.quit()

## [Mars Facts](https://space-facts.com/mars/)

In [43]:
mars_facts_url = 'https://space-facts.com/mars/'
df_list = pd.read_html(mars_facts_url)
mars_facts_df = df_list[0]
mars_facts_df.set_index(mars_facts_df.columns[0], inplace=True)
mars_facts_df

,1
0,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [45]:
args_dct = {
    'header': False,
    'index_names': False,
    'border': 0,
    'justify': 'right',
    'classes': 'table table-striped table-sm small'
}

# Export to file
mars_facts_df.to_html('test_table.html', **args_dct)

# As a string
mars_facts_table = mars_facts_df.to_html(**args_dct)
mars_facts_table

'<table border="0" class="dataframe table table-striped table-sm small">\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

## [Mars Hemispheres](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars)

In [23]:
mars_hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
astrogeology_base_url = 'https://astrogeology.usgs.gov'
response = requests.get(mars_hemispheres_url)
soup = BeautifulSoup(response.text, 'lxml')

In [24]:
# First, iterate throught the list of hemispheres, read their names and urls to download pages
hemisphere_image_urls = []
item_nodes = soup.find_all('div', class_='item')
for item_node in item_nodes:
    anchor_node = item_node.contents[0]
    download_url = astrogeology_base_url + anchor_node['href']

    header_node = anchor_node.find('h3')
    name = ' '.join(header_node.text.split(' ')[:-1])

    hemisphere_image_urls.append({
        'title': name,
        'download_page_url': download_url
    })

In [25]:
# Secondly, load every image download page and retrieve image url
for hemisphere_dct in hemisphere_image_urls:
    time.sleep(1)
    response = requests.get(hemisphere_dct['download_page_url'])
    soup = BeautifulSoup(response.text, 'lxml')
    
    download_node = soup.find('div', class_='downloads')
    first_anchor = download_node.find('a')

    del hemisphere_dct['download_page_url']
    hemisphere_dct['img_url'] = first_anchor['href']

In [26]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

### Dictionary with test data

In [42]:
test_dct = {
    'news_title': news_title,
    'news_para': news_para,
    'featured_image_url': featured_image_url,
    'featured_image_alt': featured_image_alt,
    'mars_weather': mars_weather,
    'mars_facts_table': mars_facts_table,
    'hemisphere_image_urls': hemisphere_image_urls
}

test_dct

{'news_title': "NASA's Perseverance Rover Will Look at Mars Through These 'Eyes'",
 'news_para': 'A pair of zoomable cameras will help scientists and rover drivers with high-resolution color images.',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19113-1920x1200.jpg',
 'featured_image_alt': 'Martian Concretions Near Fram Crater',
 'mars_weather': 'InSight sol 508 (2020-05-01) low -92.2ºC (-134.0ºF) high -2.4ºC (27.7ºF) winds from the SW at 5.1 m/s (11.3 mph) gusting to 15.8 m/s (35.3 mph) pressure at 6.80 hPa',
 'mars_facts_table': '<table border="0" class="dataframe table table-striped">\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orb